<a href="https://colab.research.google.com/github/annaiesenfeld/deltavsbacchus/blob/main/Image_Analysis_CompAssays.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial Setup 

In [ ]:
!git clone https://gitlab.com/dunloplab/delta.git

!pip install -r delta/requirements.txt
!pip install python-bioformats

In [ ]:

import sys
sys.path.append('/content/delta')

import delta

In [ ]:
delta.assets.download_assets(
    load_models=True,
    load_sets=False,
    load_evals=False,
    config_level='global'
)

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive2/', force_remount=True)

# To download files to your local drive:
from google.colab.files import download as colab_download
from google.colab.files import upload as colab_upload

# To display resuults movies:
from IPython.display import HTML
from base64 import b64encode

def display_movie(filename):
  mp4 = open(filename,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML("""
  <video width=600 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url
  )

In [ ]:
# code to calculate crowdedness during a division event 
import math
import numpy as np

def get_colony(cells, mother_id):
  colony = []
  colony.append(mother_id)
  for cell in lin.cells:
    if cell['mother'] == mother_id or cell['mother'] in colony:
      colony.append(cell['id'])
                    
  return colony 


def get_centre(npole, opole): # get cell centre, using the coordinates between two poles of the cell 
  cell_centre = ((npole[0] + opole[0])/2 ,  (npole[1] + opole[1])/2)
  return cell_centre

def get_cell_dist(centre1, centre2): # get the distance between two cell centres 
  # dist = ((centre1[0] + centre2[0])/2 ,  (centre1[1] + centre2[1])/2)
  dist = math.sqrt(((centre1[0] - centre2[0])**2 + (centre1[1] - centre2[1])**2))
  return dist

def get_crowd_2(cell_dist, cell_length):
  crowd = math.e**((-cell_dist/cell_length))
  return crowd

# not useful - use centre 
def is_neighbour(centre1, c2_opole, c2_npole, cell1_l0):
  dist_opole = math.sqrt(((centre1[0] - c2_opole[0])**2 + (centre1[1] - c2_opole[1])**2))
  dist_npole = math.sqrt(((centre1[0] - c2_npole[0])**2 + (centre1[1] - c2_npole[1])**2))
  if dist_opole < 2*cell1_l0 or dist_npole < 2*cell1_l0: 
    return True 
  else: 
    return False 

def is_in_radius(centre1,cell1_l0,c2_opole, c2_npole): 
  radius = cell1_l0 * 2 
  return(((c2_opole[0] - centre1[0])**2 + (c2_opole[1] - centre1[1])**2) < radius **2 or  ((c2_npole[0] - centre1[0])**2 + (c2_npole[1] - centre1[1])**2) < radius **2)
  
def get_length_alt(npole, opole):
  cell_length = math.sqrt((opole[0]- npole[0])**2 + (opole[1]- npole[1])**2)
  return cell_length

def get_poleside(centre1, opole1, npole1, npole2, opole2, cell1_length, cell1_width):
  # substracting the coordinates of the centre of cell1 ( the cell in question) from the poles of all cells 
  # After this, the centre of cell 1 lies at 0,0 
  opole1_new = [opole1[0] - centre1[0], opole1[1] - centre1[1]]
  npole1_new = [npole1[0] - centre1[0] , npole1[1] - centre1[1]]
  npole2_new = [npole2[0] - centre1[0] , npole2[1] - centre1[1]]
  opole2_new = [opole2[0] - centre1[0] , opole2[1] - centre1[1]]
  # tan theta for cell 1: y(npole)- y(opole) / x(npole) -x(opole)
  # opole and npole is just delta nomenclature for the 2 poles 
  tantheta = (opole1[1]- npole1[1]) / (opole1[0]- npole1[0])
  # using arctan to get theta 
  theta = math.atan(tantheta)
  #  creating a rotation matrix, rot using the theta obtained 
  rot = np.array([[math.cos(theta), -math.sin(theta)], [math.sin(theta), math.cos(theta)]])
  # multiplying each coordinate by rotation matrix to perform rotation
  opole1_rot = np.dot(rot, opole1_new)
  npole1_rot = np.dot(rot, npole1_new)
  opole2_rot = np.dot(rot, opole2_new)    
  npole2_rot = np.dot(rot, npole2_new)
  # now for each pole of cell 2 : 
  # first checking that the x coordinate lies within the length of cell 1 
  if abs(npole2_rot[0]) < cell1_length:
    # if that is the case, checking that the y coordinate is within 2w of cell1 
    if  abs(npole2_rot[1]) < 2*cell1_width:
      return True 
  elif abs(opole2_rot[0]) < cell1_length:
    if  abs(opole2_rot[1]) < 2*cell1_width:
      return True

# flipping the turn function in order to count the pole to side contacts but in the frame of cell 1 
def get_poleside_flipped(centre2, opole1, npole1, npole2, opole2, cell2_length, cell2_width):
  # substracting the coordinates of the centre of cell1 ( the cell in question) from the poles of all cells 
  # After this, the centre of cell 1 lies at 0,0 
  opole1_new = [opole1[0] - centre2[0], opole1[1] - centre2[1]]
  npole1_new = [npole1[0] - centre2[0] , npole1[1] - centre2[1]]
  npole2_new = [npole2[0] - centre2[0] , npole2[1] - centre2[1]]
  opole2_new = [opole2[0] - centre2[0] , opole2[1] - centre2[1]]
  # tan theta for cell 1: y(npole)- y(opole) / x(npole) -x(opole)
  # opole and npole is just delta nomenclature for the 2 poles 
  tantheta = (opole2[1]- npole2[1]) / (opole2[0]- npole2[0])
  # using arctan to get theta 
  theta = math.atan(tantheta)
  #  creating a rotation matrix, rot using the theta obtained 
  rot = np.array([[math.cos(theta), -math.sin(theta)], [math.sin(theta), math.cos(theta)]])
  # multiplying each coordinate by rotation matrix to perform rotation
  opole1_rot = np.dot(rot, opole1_new)
  npole1_rot = np.dot(rot, npole1_new)
  opole2_rot = np.dot(rot, opole2_new)    
  npole2_rot = np.dot(rot, npole2_new)
  # now for each pole of cell 2 : 
  # first checking that the x coordinate lies within the length of cell 1 
  if abs(npole1_rot[0]) < cell2_length:
    # if that is the case, checking that the y coordinate is within 2w of cell1 
    if  abs(npole1_rot[1]) < 2*cell2_width:
      return True 
  elif abs(opole1_rot[0]) < cell2_length:
    if  abs(opole1_rot[1]) < 2*cell2_width:
      return True
 
 #Colony refinement 
def refine_colony(colony):
  for cnb in colony: 
    cell = lin.cells[cnb]
    npole1 = cell['new_pole'][-1]
    opole1 = cell['old_pole'][-1]
    # get centre of cell 1 for distance function 
    cell1_centre = get_centre(npole1, opole1)
    cell1_length = get_length_alt(npole1, opole1)
    cell1_width = cell['width'][0]
    neighbours = 0
    for cnb2 in lin.cellnumbers[-1]:
      if cnb2 not in colony: 
        cell2 = lin.cells[cnb2]
        npole2 = cell2['new_pole'][-1]
        opole2 = cell2['old_pole'][-1]
        cell2_centre = get_centre(npole2, opole2)
        cell2_length = cell2['length'][0]
        cell2_width = cell2['width'][0]
      if (get_cell_dist(opole1, npole2) < (2*cell1_width)) or (get_cell_dist(opole1, opole2) < (2*cell1_width)) or (get_cell_dist(npole1, opole2) < (2*cell1_width)) or (get_cell_dist(npole1, npole2) < (2*cell1_width)): 
        if cnb2 not in colony: 
          colony.append(cnb2)
      
  return(colony)

  


#Sample Setup 

In [ ]:

delta.config.load_config(presets='2D')

#these two lines might fix tracking issues: (only when actually running the pipeline)
delta.config.drift_correction = True
delta.config.whole_frame_drift = True

# # '/content/gdrive/MyDrive/A_Gm_Rep1_Ph_run/A_Gm_Rep1_unzipped',
#  # resfolder='/content/gdrive/MyDrive/A_Gm_Rep1_Ph_run/A_Gm_Rep1_eval_movie2/'

reader = delta.utilities.xpreader(
   '/content/gdrive2/.shortcut-targets-by-id/1PFWG3Y6nCG2LwgS7ZYW9SVyGoRCzbaZI/Delta2Runs/vsFpvB_gm_RepBio1_RepT2_tl/unzipped',
    # prototype='Position%01d_Channel%01d_Frame%00002d.tif',
)

processor = delta.pipeline.Pipeline(
    reader,
    resfolder='/content/gdrive2/.shortcut-targets-by-id/1PFWG3Y6nCG2LwgS7ZYW9SVyGoRCzbaZI/Delta2Runs/vsFpvB_gm_RepBio1_RepT2_tl/eval_movie',
    reload=True
    )

In [ ]:
lin = processor.positions[0].rois[0].lineage


#Image Analysis

In [ ]:
import os
import re
import pandas as pd
from tifffile import imread
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
# get image dimensions in pixels 
from PIL import Image
filepath = '/content/gdrive2/.shortcut-targets-by-id/1PFWG3Y6nCG2LwgS7ZYW9SVyGoRCzbaZI/Delta2Runs/vsFpvB_gm_RepBio1_RepT2_tl/unzipped/Position01_Channel01_Frame000008.tif'
img = Image.open(filepath)
  
# get width and height
width = img.width
height = img.height
  
# display width and height
print("The height of the image is: ", height)
print("The width of the image is: ", width)


In [ ]:
import numpy as np

def create_cell_mask(w,h,centre, radius): 
  # w = width of image
  # h = height of image 
  # n = the frame you want to look at in a given moment 
  Y, X = np.ogrid[:h, :w]
  dist_from_center = np.sqrt((X - centre[0])**2 + (Y-centre[1])**2)
  mask = dist_from_center <= radius
  return mask

  

In [ ]:
print(lin.cells[48]['new_pole'])

In [ ]:


delta.utilities.cell_fluo(GFP_folder, )

In [ ]:
co_ch = {'wt':[], 'mnt':[]} # initialise dictionary of wt and mutant lineage ids
GFP_folder = '/content/gdrive2/.shortcut-targets-by-id/1sR73S8JfI8yRgqG052uxjiO46rLG1Hho/vsFpvB_gm_RepBio1_RepT2/GFP'
for gfp in os.listdir(GFP_folder):
  if re.search(r'T=31', gfp):
    gfp_img_path = GFP_folder+'/'+ gfp
    # print(gfp_img_path)

RFP_folder = '/content/gdrive2/.shortcut-targets-by-id/1sR73S8JfI8yRgqG052uxjiO46rLG1Hho/vsFpvB_gm_RepBio1_RepT2/RFP'
for rfp in os.listdir(RFP_folder):
  if re.search(r'T=31', rfp):
    rfp_img_path = RFP_folder+'/'+ rfp
    # print(rfp_img_path)

imageGfp = imread(gfp_img_path) # open RFP and GFP images
imageRfp = imread(rfp_img_path)

for c in lin.cellnumbers[30]: 
  # print(c)
  cell = lin.cells[c]
  j= cell['frames'].index(30)
  cell_npole = cell['new_pole'][j]
  cell_opole = cell['old_pole'][j]
  cell_centre = get_centre(cell_npole, cell_opole)
  # print(cell_centre)
  cell_radius = (cell['width'][0])
  mask = create_cell_mask(2048,2048, cell_centre, cell_radius )

  gfp_intensity, rfp_intensity = sum(imageGfp[mask]), sum(imageRfp[mask])
  if rfp_intensity > gfp_intensity :
    if c not in co_ch['mnt']:
      co_ch['mnt'].append(c)
      print(str(c), 'mnt')
  if gfp_intensity > rfp_intensity:
    if c not in co_ch['wt']:
      co_ch['wt'].append(c)
      # print(str(c),'wt')

In [ ]:
# check that colony looks good 

import numpy as np


labels = processor.positions[0].rois[0].label_stack[-1]

# Initialize color image (all light gray)
color_image = np.full(
    shape=labels.shape+(3,),
    fill_value=175,
    dtype=np.uint8
    )

# colony1 = get_colony(lin.cells, 1)
# colony2 = get_colony(lin.cells, 2)

# colony = colony1 + colony2


# col_1011_cids, col_1011_neighbours, col_1011_cycles = numneighbours_immediate(colony_1011)

for cell in lin.cellnumbers[30]:
  if cell in co_ch['mnt']:
    color = [255,0,0]
  elif cell in co_ch['wt']:
    color = [0,255,0]
  else:
     color = [0,0,0]


  for c, val in enumerate(color):
      color_image[:,:,c][labels==cell+1]=val
      



plt.figure(figsize = (15,15))
plt.imshow(color_image)

In [ ]:
import seaborn as sns
import statistics as stats

first_cells = lin.cells[0]
print(first_cells)

pop_birth_lengths=[]
for cell in lin.cells: 
  if cell['id'] in co_ch['mnt']:
    if cell['id'] in first_cells or cell['mother'] is not None: 
      birth_length = cell['length'][0]
      pop_birth_lengths.append(birth_length)


plt.figure(figsize=(7,7))
ax2 = sns.distplot(pop_birth_lengths, hist=True, kde=True, 
             bins=int(len(pop_birth_lengths)/50), color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})
ax2.set_ylabel('Density', fontname = 'STIXGeneral', fontsize = 23)
ax2.set_xlabel('Birth length (pixels)', fontname = 'STIXGeneral', fontsize = 23)
ax2.set_title ('Birth length distribution in $\Delta$FpvB P.aureginosa cells', fontname = 'STIXGeneral', fontsize = 23, pad = 20)
# ax.set_xticklabels(fontname = 'STIXGeneral', fontsize=20)
# ax.set_yticks(fontname = 'STIXGeneral', fontsize=20)
ylabels = plt.yticks(fontname = 'STIXGeneral', fontsize=22)
xlabels = plt.xticks(fontname = 'STIXGeneral', fontsize=22)
plt.show()

print(stats.mean(pop_birth_lengths), stats.stdev(pop_birth_lengths))



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns 
from matplotlib import rc
import statistics as stat
from matplotlib.pyplot import figure

lin = processor.positions[0].rois[0].lineage

first_cells = lin.cellnumbers[0]

pop_div_lengths = []
div_frames = []
pop_birth_lengths = []

for cell in lin.cells: 
  if cell['id'] in co_ch['mnt']:
    daughters = [cell['daughters'].index(daughter) for daughter in cell['daughters'] if daughter is not None]
    for daughter in daughters: 
      if daughters.index(daughter) == 0: 
        div_len_idx = daughter - 1 
        div_length = cell['length'][div_len_idx] 
        div_frame = cell['frames'][div_len_idx] 
        pop_div_lengths.append(div_length)
        pop_birth_lengths.append(cell['length'][0])
        div_frames.append(div_frame)
      
        


plt.figure(figsize=(7,7))
ax = sns.distplot(pop_div_lengths, hist=True, kde=True, 
             bins=int(len(pop_div_lengths)/50), color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})
ax.set_ylabel('Density', fontname = 'STIXGeneral', fontsize = 23)
ax.set_xlabel('Division length (pixels)', fontname = 'STIXGeneral', fontsize = 23)
ax.set_title ('Division length distribution in $\Delta$FpvB P.aureginosa cells', fontname = 'STIXGeneral', fontsize = 23, pad = 20)
# ax.set_xticklabels(fontname = 'STIXGeneral', fontsize=20)
# ax.set_yticks(fontname = 'STIXGeneral', fontsize=20)
ylabels = plt.yticks(fontname = 'STIXGeneral', fontsize=22)
xlabels = plt.xticks(fontname = 'STIXGeneral', fontsize=22)
plt.show()

print(stats.mean(pop_div_lengths), stats.stdev(pop_div_lengths))

# plt.scatter(div_frames,pop_div_lengths)
# plt.xlabel('Division frame', fontname = 'STIXGeneral', fontsize=20)
# plt.ylabel('Division length (pixels)', fontname = 'STIXGeneral',  fontsize=20)
# # plt.title('Division length in WT P.aureginosa as a function of division frame')
# plt.title('Division length in WT P.aureginosa as a function of division frame',fontname = 'STIXGeneral', fontsize=20, pad =20)
# plt.xticks(fontname = 'STIXGeneral', fontsize=20)
# plt.yticks(fontname = 'STIXGeneral', fontsize=20)
# figure(figsize=(20,16))
# plt.show()


In [ ]:
co_ch = {'wt':[], 'mnt':[]} # initialise dictionary of wt and mutant lineage ids
GFP_folder = '/content/gdrive2/.shortcut-targets-by-id/1sR73S8JfI8yRgqG052uxjiO46rLG1Hho/vsFpvB_gm_RepBio1_RepT2/GFP'
for gfp in os.listdir(GFP_folder):
  if re.search(r'T=20', gfp):
    gfp_img_path = GFP_folder+'/'+ gfp
    # print(gfp_img_path)

RFP_folder = '/content/gdrive2/.shortcut-targets-by-id/1sR73S8JfI8yRgqG052uxjiO46rLG1Hho/vsFpvB_gm_RepBio1_RepT2/RFP'
for rfp in os.listdir(RFP_folder):
  if re.search(r'T=20', rfp):
    rfp_img_path = RFP_folder+'/'+ rfp
    # print(rfp_img_path)

imageGfp = imread(gfp_img_path) # open RFP and GFP images
imageRfp = imread(rfp_img_path)

for c in lin.cellnumbers[20]: 
  # print(c)
  cell = lin.cells[c]
  j= cell['frames'].index(20)
  cell_npole = cell['new_pole'][j]
  cell_opole = cell['old_pole'][j]
  cell_centre = get_centre(cell_npole, cell_opole)
  # print(cell_centre)
  cell_radius = (cell['width'][0])
  mask = create_cell_mask(2048,2048, cell_centre, cell_radius )
  fluo = delta.utilities.cell_fluo(, mask)


In [ ]:



co_ch = {'wt':[], 'mnt':[]} # initialise dictionary of wt and mutant lineage ids
GFP_folder = '/content/gdrive2/.shortcut-targets-by-id/1sR73S8JfI8yRgqG052uxjiO46rLG1Hho/vsFpvB_gm_RepBio1_RepT2/GFP'
RFP_folder = '/content/gdrive2/.shortcut-targets-by-id/1sR73S8JfI8yRgqG052uxjiO46rLG1Hho/vsFpvB_gm_RepBio1_RepT2/RFP'



for i in range(0,29):
  print('frame:'+ str(i))
  for gfp in os.listdir(GFP_folder):
    if re.search(r'T=%i', gfp):
      gfp_img_path = GFP_folder+'/'+ gfp
      print(gfp_img_path)


  for rfp in os.listdir(RFP_folder):
    if re.search(r'T=%i', rfp):
      rfp_img_path = RFP_folder+'/'+ rfp
      print(rfp_img_path)

  imageGfp = imread(gfp_img_path) # open RFP and GFP images
  imageRfp = imread(rfp_img_path)
  
  for c in lin.cellnumbers[i]: 
    # print(c)
    cell = lin.cells[c]
    j= cell['frames'].index(i)
    cell_npole = cell['new_pole'][j]
    cell_opole = cell['old_pole'][j]
    cell_centre = get_centre(cell_npole, cell_opole)
    # print(cell_centre)
    cell_radius = (cell['width'][0])
    mask = create_cell_mask(2048,2048, cell_centre, cell_radius )

    

    gfp_intensity, rfp_intensity = sum(imageGfp[mask]), sum(imageRfp[mask])
    if rfp_intensity > gfp_intensity/5:
      if c not in co_ch['mnt']:
        co_ch['mnt'].append(c)
        print(str(c), 'mnt')
    elif gfp_intensity/5 > rfp_intensity:
      if c not in co_ch['wt']:
        co_ch['wt'].append(c)
        print(str(c),'wt')
        


In [ ]:

# co_ch = {'wt':[], 'mnt':[]} # initialise dictionary of wt and mutant lineage ids
# GFP_folder = '/content/gdrive2/.shortcut-targets-by-id/1sR73S8JfI8yRgqG052uxjiO46rLG1Hho/vsFpvB_gm_RepBio1_RepT2/GFP'
# for gfp in os.listdir(GFP_folder):
#   if re.search(r'T=20', gfp):
#     gfp_img_path = GFP_folder+'/'+ gfp
#     # print(gfp_img_path)

# RFP_folder = '/content/gdrive2/.shortcut-targets-by-id/1sR73S8JfI8yRgqG052uxjiO46rLG1Hho/vsFpvB_gm_RepBio1_RepT2/RFP'
# for rfp in os.listdir(RFP_folder):
#   if re.search(r'T=20', rfp):
#     rfp_img_path = RFP_folder+'/'+ rfp
#     # print(rfp_img_path)

# imageGfp = imread(gfp_img_path) # open RFP and GFP images
# imageRfp = imread(rfp_img_path)

# for c in lin.cellnumbers[20]: 
#   # print(c)
#   cell = lin.cells[c]
#   j= cell['frames'].index(20)
#   cell_npole = cell['new_pole'][j]
#   cell_opole = cell['old_pole'][j]
#   cell_centre = get_centre(cell_npole, cell_opole)
#   # print(cell_centre)
#   cell_radius = (cell['width'][0])
#   mask = create_cell_mask(2048,2048, cell_centre, cell_radius )

#   gfp_intensity, rfp_intensity = sum(imageGfp[mask]), sum(imageRfp[mask])
#   if rfp_intensity > gfp_intensity/2 :
#     if c not in co_ch['mnt']:
#       co_ch['mnt'].append(c)
#       print(str(c), 'mnt')
#   if gfp_intensity/3 > rfp_intensity:
#     if c not in co_ch['wt']:
#       co_ch['wt'].append(c)
#       print(str(c),'wt')


In [ ]:
print(len(lin.cellnumbers))

In [ ]:
# check that colony looks good 

import numpy as np


labels = processor.positions[0].rois[0].label_stack[-1]

# Initialize color image (all light gray)
color_image = np.full(
    shape=labels.shape+(3,),
    fill_value=175,
    dtype=np.uint8
    )

# colony1 = get_colony(lin.cells, 1)
# colony2 = get_colony(lin.cells, 2)

# colony = colony1 + colony2


# col_1011_cids, col_1011_neighbours, col_1011_cycles = numneighbours_immediate(colony_1011)

for cell in lin.cellnumbers[24]:
  color = [0,0,0]
  # if cell in co_ch['mnt']:
  #   color = [0,0,255]
  # elif cell in co_ch['wt']:
  #   color = [0,255,0]
  # else:
  #    color = [0,0,0]


  for c, val in enumerate(color):
      color_image[:,:,c][labels==cell+1]=val
      



plt.figure(figsize = (15,15))
plt.imshow(color_image)

In [ ]:
# check that colony looks good 

import numpy as np


labels = processor.positions[0].rois[0].label_stack[-1]

# Initialize color image (all light gray)
color_image = np.full(
    shape=labels.shape+(3,),
    fill_value=175,
    dtype=np.uint8
    )

# colony1 = get_colony(lin.cells, 1)
# colony2 = get_colony(lin.cells, 2)

# colony = colony1 + colony2


# col_1011_cids, col_1011_neighbours, col_1011_cycles = numneighbours_immediate(colony_1011)

for cell in lin.cellnumbers[20]:
  if cell in co_ch['mnt']:
    color = [0,0,255]
  elif cell in co_ch['wt']:
    color = [0,255,0]
  else:
     color = [0,0,0]


  for c, val in enumerate(color):
      color_image[:,:,c][labels==cell+1]=val
      



plt.figure(figsize = (15,15))
plt.imshow(color_image)

In [ ]:
# check that colony looks good 

import numpy as np


labels = processor.positions[0].rois[0].label_stack[-1]

# Initialize color image (all light gray)
color_image = np.full(
    shape=labels.shape+(3,),
    fill_value=175,
    dtype=np.uint8
    )

# colony1 = get_colony(lin.cells, 1)
# colony2 = get_colony(lin.cells, 2)

# colony = colony1 + colony2


# col_1011_cids, col_1011_neighbours, col_1011_cycles = numneighbours_immediate(colony_1011)

for cell in lin.cellnumbers[-2]:
  if cell in co_ch['mnt']:
    color = [0,0,255]
  elif cell in co_ch['wt']:
    color = [0,255,0]
  else:
     color = [0,0,0]


  for c, val in enumerate(color):
      color_image[:,:,c][labels==cell+1]=val
      



plt.figure(figsize = (15,15))
plt.imshow(color_image)

In [ ]:
#  create copy of lin.cells for modification 
import copy

lin.cells2 = copy.deepcopy(lin.cells)

In [ ]:
import os 
GFP_folder = '/content/gdrive2/.shortcut-targets-by-id/1sR73S8JfI8yRgqG052uxjiO46rLG1Hho/vsFpvB_gm_RepBio1_RepT2/GFP'
for gfp in os.listdir(GFP_folder):
  if re.search(r'T=0\b', gfp):
    gfp_img_path = GFP_folder+'/'+ gfp
    print(gfp_img_path)
    break

  #  T_ = file.split('T=',2)
  #  T = T_[1][0]+T_[1][1]
   
imageGfp = imread(gfp_img_path)
  



In [ ]:
imageGfp

In [ ]:
# get centre 

for cnb in lin.cellnumbers[0]:
  cell = lin.cells2[cnb]
  cell_npole = cell['new_pole'][0]
  cell_opole = cell['old_pole'][0]
  cell_centre = get_centre(cell_npole, cell_opole)
  print(cell_centre)




# Image analysis but for vsA 

In [ ]:
delta.config.load_config(presets='2D')

#these two lines might fix tracking issues: (only when actually running the pipeline)
delta.config.drift_correction = True
delta.config.whole_frame_drift = True

# # '/content/gdrive/MyDrive/A_Gm_Rep1_Ph_run/A_Gm_Rep1_unzipped',
#  # resfolder='/content/gdrive/MyDrive/A_Gm_Rep1_Ph_run/A_Gm_Rep1_eval_movie2/'

reader = delta.utilities.xpreader(
   '/content/gdrive2/.shortcut-targets-by-id/1PFWG3Y6nCG2LwgS7ZYW9SVyGoRCzbaZI/Delta2Runs/vsFpvA_gm_RepBio1_RepT2_tl/unzipped_images',
    # prototype='Position%01d_Channel%01d_Frame%00002d.tif',
)

processor = delta.pipeline.Pipeline(
    reader,
    resfolder='/content/gdrive2/.shortcut-targets-by-id/1PFWG3Y6nCG2LwgS7ZYW9SVyGoRCzbaZI/Delta2Runs/vsFpvA_gm_RepBio1_RepT2_tl/eval_movie',
    reload=True
    )

In [ ]:
lin = processor.positions[0].rois[0].lineage

In [ ]:
for cell in lin.cells: 
  print(cell['width'])

In [ ]:
co_ch = {'wt':[], 'mnt':[]} # initialise dictionary of wt and mutant lineage ids

for i in range(0,29):
  print('frame:'+ str(i))
  GFP_folder = '/content/gdrive2/.shortcut-targets-by-id/1PFWG3Y6nCG2LwgS7ZYW9SVyGoRCzbaZI/Delta2Runs/vsFpvA_gm_RepBio1_RepT2_tl/Rep2/GFP'
  for gfp in os.listdir(GFP_folder):
    if re.search(r'T=%i', gfp):
      gfp_img_path = GFP_folder+'/'+ gfp
      # print(gfp_img_path)

  RFP_folder = '/content/gdrive2/.shortcut-targets-by-id/1PFWG3Y6nCG2LwgS7ZYW9SVyGoRCzbaZI/Delta2Runs/vsFpvA_gm_RepBio1_RepT2_tl/Rep2/RFP'
  for rfp in os.listdir(RFP_folder):
    if re.search(r'T=%i', rfp):
      rfp_img_path = RFP_folder+'/'+ rfp
      print(rfp_img_path)

  
  for c in lin.cellnumbers[i]: 
    # print(c)
    cell = lin.cells[c]
    j= cell['frames'].index(i)
    cell_npole = cell['new_pole'][j]
    cell_opole = cell['old_pole'][j]
    cell_centre = get_centre(cell_npole, cell_opole)
    # print(cell_centre)
    cell_radius = (cell['width'][0])
    mask = create_cell_mask(2048,2048, cell_centre, 6)

    imageGfp = imread(gfp_img_path) # open RFP and GFP images
    imageRfp = imread(rfp_img_path)

    gfp_intensity, rfp_intensity = sum(imageGfp[mask]), sum(imageRfp[mask])
    print(gfp_intensity, rfp_intensity)
    if rfp_intensity > gfp_intensity :
      print(str(c), 'mnt')
      if c not in co_ch['mnt']:
        co_ch['mnt'].append(c)
    if gfp_intensity > rfp_intensity:
      if c not in co_ch['wt']:
        co_ch['wt'].append(c)
        print(str(c),'wt')
  

In [ ]:
# check that colony looks good 

import numpy as np


labels = processor.positions[0].rois[0].label_stack[-1]

# Initialize color image (all light gray)
color_image = np.full(
    shape=labels.shape+(3,),
    fill_value=175,
    dtype=np.uint8
    )

# colony1 = get_colony(lin.cells, 1)
# colony2 = get_colony(lin.cells, 2)

# colony = colony1 + colony2


# col_1011_cids, col_1011_neighbours, col_1011_cycles = numneighbours_immediate(colony_1011)

for cell in lin.cellnumbers[-2]:
  if cell in co_ch['mnt']:
    color = [0,0,255]
  elif cell in co_ch['wt']:
    color = [0,255,0]
  else:
     color = [0,0,0]


  for c, val in enumerate(color):
      color_image[:,:,c][labels==cell+1]=val
      



plt.figure(figsize = (15,15))
plt.imshow(color_image)

In [ ]:
print(len(co_ch['mnt']))

In [ ]:
GFP_folder = '/content/gdrive2/.shortcut-targets-by-id/1qthuP1OIiIg1Lnw7lfFtx1ueNMHZemtI/FpvA_gm_RepBio1_Rep2T/GFP'
for gfp in os.listdir(GFP_folder):
  if re.search(r'T=30', gfp):
    gfp_img_path = GFP_folder+'/'+ gfp
    # print(gfp_img_path)

RFP_folder = '/content/gdrive2/.shortcut-targets-by-id/1qthuP1OIiIg1Lnw7lfFtx1ueNMHZemtI/FpvA_gm_RepBio1_Rep2T/RFP'
for rfp in os.listdir(RFP_folder):
  if re.search(r'T=30', rfp):
    rfp_img_path = RFP_folder+'/'+ rfp
    # print(rfp_img_path)

imageGfp = imread(gfp_img_path) # open RFP and GFP images
imageRfp = imread(rfp_img_path)

for c in lin.cellnumbers[-1]: 
  # print(c)
  cell = lin.cells[c]
  # j= cell['frames'].index(i)
  cell_npole = cell['new_pole'][-1]
  cell_opole = cell['old_pole'][-1]
  cell_centre = get_centre(cell_npole, cell_opole)
  # print(cell_centre)
  cell_radius = (cell['width'][0])
  mask = create_cell_mask(2048,2048, cell_centre, cell_radius )

  gfp_intensity, rfp_intensity = sum(imageGfp[mask]), sum(imageRfp[mask])
  print(gfp_intensity, rfp_intensity)
  if rfp_intensity > gfp_intensity:
      print(str(c), 'mnt')
  if gfp_intensity > rfp_intensity:
      # print(str(c),'wt')
      print('/')
